In [68]:
from textblob import TextBlob
import sys
import tweepy
import json
import requests
import pandas as pd
from IPython.display import display, HTML
consumerkey=""
consumersecret=""
accesstoken=""
accesstokensecret=""
auth=tweepy.OAuthHandler(consumerkey,consumersecret)
auth.set_access_token(accesstoken,accesstokensecret)
api=tweepy.API(auth)

In [70]:
#Fetching Tweets about Ferrari (Only 200 tweets were allowed to fetch)
noofsearchterm = int(input("enter the no of tweets to analyse:"))
print("Analysing "+ str(noofsearchterm) +" tweets for Ferrari")
searchterm = "ferrari"
tweets=tweepy.Cursor(api.search,q=searchterm,lang="en").items(noofsearchterm)
positive=0
negative=0
neutral=0
polarity=0
tweet_list=[]
for tweet in tweets:
    tweet_info=dict()
    imgs=[]
    tweet_info['Twitter_Text']=tweet.text
    tweet_info['Date/Time']=str(tweet.created_at)
    tweet_info['No._of_Likes']=tweet.favorite_count
    tweet_info['No._of_Retweets']=tweet.retweet_count
    
    #checking for images in tweets
    for i in tweet.entities.get("media",[{}]):
        if i.get("type",None) == "photo":
            image_content=requests.get(i["media_url"])
            imgs.append(image_content)
            tweet_info['No._of_Images']=len(imgs)
        else:
            tweet_info['No._of_Images']="None"
    
    #sentiment analysis
    analysis=TextBlob(tweet.text)
    polarity=analysis.sentiment.polarity
    if(polarity==0):
        tweet_info['Sentiment']="neutral"
    elif(polarity<0):
        tweet_info['Sentiment']="negative"
    elif(polarity>0):
        tweet_info['Sentiment']="positive"
    
    #dumping tweets information into json file
    tweet_list.append(tweet_info)
    file_info=open('tweets.json','w')
    json.dump(tweet_list,file_info,indent=4,sort_keys=True)
    file_info.close()

#reading json file and displaying in tabular form
rd = pd.read_json(r'tweets.json')
df=pd.DataFrame(rd)
display(HTML(df.to_html()))

enter the no of tweets to analyse:200
Analysing 200 tweets for Ferrari


,Date/Time,No._of_Images,No._of_Likes,No._of_Retweets,Sentiment,Twitter_Text
0,2019-12-26 10:39:39,None,0,0,neutral,@gfx_girls @Fern_Ferrari @Brazzers contact her.
1,2019-12-26 10:39:34,1,0,4568,neutral,RT @CUNextTuesday17: Twitter going nuts trying...
2,2019-12-26 10:39:33,None,0,24,positive,RT @cindygot7king: Congratulations GOT7 - JUST...
3,2019-12-26 10:39:01,None,0,405,negative,RT @bamcode970502: [HQ] 191225 SBS\n\n@BamBam1...
4,2019-12-26 10:38:45,None,0,38,positive,RT @cindygot7king: #IGOT7 ATTENTION ATTENTION🙏...
5,2019-12-26 10:38:08,None,0,0,negative,"""‘Everyday Supercar’: A New Corvette Puts a Ta..."
6,2019-12-26 10:37:51,None,0,0,negative,‘Everyday Supercar’: A New Corvette Puts a Tar...
7,2019-12-26 10:37:50,None,0,9,positive,RT @10thLegio: In 1902 an Italian man digging ...
8,2019-12-26 10:37:33,1,0,0,positive,Ferrari: Letting Vettel win F1's Singapore Gra...
9,2019-12-26 10:37:33,1,0,0,positive,Ferrari: Letting Vettel win F1's Singapore Gra...
